##### 1.  Look up the Adam optimization functions in PyTorch https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other optimization function with the diabetes dataset shown in class. How does the model perform with the new optimizer? Did it perform better or worse than Adam? Why do you think that is?

The Adam optimizer is a popular choice because it adapts the learning rate simultaneously with gradient descent from what I understand.  I tried the Adamax optimizer, and it performed slightly better for this dataset than Adam.  The overall accuracy score was the same for both optimizers, but the sensitivity was 13% higher for Adamax than for Adam.  Because our biggest concern with this dataset is accurately detecting those with diabetes, sensitivity is the most important metric.  From  what we researched, it sounds like Adamax performed better because its algorithm is not influenced by gradient noise as much as the Adam algorithm.  It uses more sparse parameters, which generate less noise when applying gradient descent to the process.

In [41]:
import pandas as pd
import torch

diabetes = pd.read_csv('diabetes.csv')
diabetes_df = pd.DataFrame(diabetes)
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [43]:
import torch.nn as nn
import torch.nn.functional as F #where the activation functions are

#create tensors = matrices 
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[ 0.9314,  2.0179,  0.7807,  ...,  0.4315, -0.3748,  0.6321],
        [ 0.6326, -1.1486,  0.4654,  ..., -0.1198, -0.2942,  0.7170],
        [-0.5625, -0.4769, -0.2703,  ..., -0.2096,  2.7452,  0.0381],
        ...,
        [-0.8613, -0.7648,  0.0450,  ...,  0.7648, -0.7838, -0.3014],
        [ 0.6326,  2.2099,  1.2010,  ...,  0.4315, -0.6047,  2.7537],
        [ 0.0351,  0.7385, -0.5856,  ..., -0.3378, -0.5778,  0.2927]])


In [44]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirect reference. So, it isolates changes
        # and makes sure that children in the layers of multiple inheritence are calling
        #the right parents
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [45]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adamax(ann.parameters(),lr=0.01)
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

Epoch number: 1 with loss: 0.647470235824585
Epoch number: 11 with loss: 0.5535582304000854
Epoch number: 21 with loss: 0.49165403842926025
Epoch number: 31 with loss: 0.45121264457702637
Epoch number: 41 with loss: 0.4320707321166992
Epoch number: 51 with loss: 0.4135775864124298
Epoch number: 61 with loss: 0.3985571563243866
Epoch number: 71 with loss: 0.3826017379760742
Epoch number: 81 with loss: 0.36624687910079956
Epoch number: 91 with loss: 0.3504957854747772
Epoch number: 101 with loss: 0.3354971706867218
Epoch number: 111 with loss: 0.32112744450569153
Epoch number: 121 with loss: 0.3066723644733429
Epoch number: 131 with loss: 0.29085466265678406
Epoch number: 141 with loss: 0.2746669054031372
Epoch number: 151 with loss: 0.25882041454315186
Epoch number: 161 with loss: 0.24144336581230164
Epoch number: 171 with loss: 0.22444197535514832
Epoch number: 181 with loss: 0.20857736468315125
Epoch number: 191 with loss: 0.19238297641277313
Epoch number: 201 with loss: 0.17724098265

In [46]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) 
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       150
           1       0.56      0.62      0.59        81

    accuracy                           0.70       231
   macro avg       0.67      0.68      0.67       231
weighted avg       0.70      0.70      0.70       231



##### 2.  Write a function that lists and counts the number of divisors for an input value.

##### Example 1:

##### Input: 5
##### Output: “There are 2 divisors: 1 and 5”

##### Example 2:
##### Input: 40
##### Output: “There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40

In [47]:
def divisors(num):
    div_list=[]
    count = 0
    for i in range(1, num):
        if num % (i) == 0:
            div_list.append(i)
            count = count + 1
        else:
            pass
    print('There are ',count, ' divisors: ',div_list)

In [48]:
divisors(40)

There are  7  divisors:  [1, 2, 4, 5, 8, 10, 20]
